In [1]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql._
import org.apache.spark.sql.types.{IntegerType, DoubleType, StringType, ArrayType, StructType, StructField}

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.feature.{StringIndexer, VectorIndexer, VectorAssembler, Normalizer}
import org.apache.spark.ml.evaluation.{RegressionEvaluator, MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.classification.{RandomForestClassifier, DecisionTreeClassifier, GBTClassifier}

import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.linalg.Vector

import org.apache.spark.mllib.tree.RandomForest
import org.apache.spark.mllib.tree.model.RandomForestModel

import org.apache.spark.mllib.tree.GradientBoostedTrees
import org.apache.spark.mllib.tree.configuration.BoostingStrategy
import org.apache.spark.mllib.tree.model.GradientBoostedTreesModel

import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}

import collection.mutable._
import java.io._
import scala.math.log

In [2]:
//sqlContext = SQLContext(sc)

Name: Syntax Error.
Message: 
StackTrace: 

## Fonctions

In [116]:
// Log Loss metric
def logloss(df:DataFrame) : Double = {
    var loglossRed1 = df.map(r => (r.getAs[Double]("indexedLabel"), r.getAs[Double]("proba"))) //probability[1]
    
    // Check if(some proba are < 0
    val neg = loglossRed1.filter{ case (y,p) => (p <= 0.0 || p >= 1.0)}
    val negCount = neg.count()
    if(negCount != 0) {
        println("!!! There so non-valid probability !!! " + negCount)
        loglossRed1 = loglossRed1.filter{ case (y,p) => (p > 0.0 && p < 1.0)}
    }
    
    val loglossRed2 =  loglossRed1.map{case (y,p) => y*log(p) + (1-y)*log(1.0-p)}
    val loglossRed  =  loglossRed2.reduce((a, b) => a+b)
    
    return -1.0 * loglossRed / df.count()
}
//println("Logloss on Training " + logloss(trainPredictions))

Name: Syntax Error.
Message: 
StackTrace: 

In [4]:
// Shrink dow extrem proba
def shrink(_value:Double, _factor:Option[Double] = Some(0.2), trunc:Double = 0.1) : Double = {
    var value = _value
    
    if(value < 0.0 + trunc) {
        value = 0.0 + trunc
    }
    else if(value > 1.0 - trunc) {
        value = 1.0 - trunc
    }
        
    if(_factor == None) {
        return value
    }
    var factor:Double = _factor.get
    
    return value * (1.0-factor) + factor/2.0
}
//println(shrink(0.5))
//println(shrink(0))
//println(shrink(1))
//println(shrink(0.95))
//println(shrink(0.05))

def shrinkDf(df:DataFrame, factor:Option[Double] = Some(0.2), trunc:Double = 0.0) : DataFrame = {
    // proba=u"[1,null,null,[0.9413866396761132,0.05861336032388664]]
    //val shrinkUdf = udf(probability => shrink(float(probability.split(",")(4)(-2), factor, trunc), DoubleType()))
    
    val coder: (Vector => Double) = (probability: Vector) => {shrink(probability(1), factor, trunc)}
    val shrinkUdf = udf(coder)
    
    //val dfShrink1 = df.withColumn("proba", df("probability"))
    //println(dfShrink1.take(1)
    //val dfShrink = dfShrink1.withColumn("proba", shrinkUdf(dfShrink1("proba")))
    val dfShrink = df.withColumn("proba", shrinkUdf(df("probability")))
    
    return dfShrink
}

In [5]:
trainPredictions.select("probability").take(5).foreach(println)

Name: Compile Error
Message: <console>:46: error: not found: value trainPredictions
              trainPredictions.select("probability").take(5).foreach(println)
              ^
StackTrace: 

In [6]:
val trainPredictionsShrink = shrinkDf(trainPredictions, Some(0.3))
trainPredictionsShrink.select("probability", "proba").take(5) //.select("proba")

Name: Compile Error
Message: <console>:49: error: not found: value trainPredictions
         val trainPredictionsShrink = shrinkDf(trainPredictions, Some(0.3))
                                               ^
StackTrace: 

## Load

In [7]:
val kaggleTrain = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").load("kaggle/train.csv")

kaggleTrain.cache()

[ID: int, target: int, v1: double, v2: double, v3: string, v4: double, v5: double, v6: double, v7: double, v8: double, v9: double, v10: double, v11: double, v12: double, v13: double, v14: double, v15: double, v16: double, v17: double, v18: double, v19: double, v20: double, v21: double, v22: string, v23: double, v24: string, v25: double, v26: double, v27: double, v28: double, v29: double, v30: string, v31: string, v32: double, v33: double, v34: double, v35: double, v36: double, v37: double, v38: int, v39: double, v40: double, v41: double, v42: double, v43: double, v44: double, v45: double, v46: double, v47: string, v48: double, v49: double, v50: double, v51: double, v52: string, v53: double, v54: double, v55: double, v56: string, v57: double, v58: double, v59: d...

In [8]:
val kaggleTest = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").load("kaggle/test.csv")

In [9]:
println("Kaggle Train count " + kaggleTrain.count())
println("Kaggle Test count  " + kaggleTest.count())

Kaggle Train count 114321
Kaggle Test count  114393


In [10]:
//println(train.schema.fields
var columnsDict = new HashMap[String,List[String]].withDefaultValue(Nil)
for( col <- kaggleTrain.schema.fields ) {
    val typeKey = col.dataType
    val colName = col.name
    
    if(colName == "ID") {
        println("We have the ID columns, type " + typeKey)
        //continue
    }
    else if(colName == "target") {
        println("We have the target columns, type " + typeKey)
        //continue
    }
    else {
        columnsDict(typeKey.toString) ::= col.name.toString
    }
    
    /*if(typeKey not in columnsDict) {
        //columnsDict[typeKey] = [col.name]
        columnsDict(typeKey) ::= col.name
    }
    else{
        //columnsDict[typeKey].append(col.name)
        columnsDict += (typeKey -> col.name)
         ("dog") ::= 
    }*/
}

println("")
for( (ct, cl) <- columnsDict) {
    println(ct + " " + cl.length)
}

We have the ID columns, type IntegerType
We have the target columns, type IntegerType
DoubleType 108
StringType 19
IntegerType 4


In [11]:
kaggleTrain.stat.freqItems(Seq("target")).show()

+----------------+
|target_freqItems|
+----------------+
|          [1, 0]|
+----------------+



In [12]:
kaggleTrain.stat.crosstab("target", "target").show()

+-------------+-----+-----+
|target_target|    0|    1|
+-------------+-----+-----+
|            1|    0|87021|
|            0|27300|    0|
+-------------+-----+-----+



## Split

In [13]:
// Split the data into train and test
val Array(train, test) = kaggleTrain.randomSplit(Array(0.6, 0.4), seed = 1234)

## Deal with missing values

In [14]:
def replacementFunction(df:DataFrame) : DataFrame = {
    val description = df.describe()
    
    val descriptionCol = description.collect()
    
    var replacementDict = new HashMap[String,Any].withDefaultValue(Nil)
    for(col <- columnsDict("DoubleType")){
        replacementDict(col) = descriptionCol(1).getAs(col)
    }
    for(col <- columnsDict("IntegerType")){
        replacementDict(col) = descriptionCol(1).getAs(col)
    }
    //println(replacementDict)
    println("Replacing!")
    return df.na.fill(replacementDict.toMap)
}

### Experimentation

In [15]:
val description = train.describe()

In [16]:
description.select("v10", "target", "summary").show()

+------------------+-------------------+-------+
|               v10|             target|summary|
+------------------+-------------------+-------+
|             68428|              68479|  count|
|1.8805752686729722| 0.7608609938813359|   mean|
|1.3981410306820936|0.42656089711962514| stddev|
| -9.87531659989E-7|                  0|    min|
|     18.5339164478|                  1|    max|
+------------------+-------------------+-------+



In [17]:
val descriptionCol = description.collect()

In [18]:
println("Mean v12 example " + descriptionCol(1).getAs("v12"))

Mean v12 example 6.879828782575871


In [19]:
columnsDict("StringType")
println("Mean v3 example" + descriptionCol(1).getAs("v3"))

Name: java.lang.IllegalArgumentException
Message: Field "v3" does not exist.
StackTrace: org.apache.spark.sql.types.StructType$$anonfun$fieldIndex$1.apply(StructType.scala:234)
org.apache.spark.sql.types.StructType$$anonfun$fieldIndex$1.apply(StructType.scala:234)
scala.collection.MapLike$class.getOrElse(MapLike.scala:128)
scala.collection.AbstractMap.getOrElse(Map.scala:58)
org.apache.spark.sql.types.StructType.fieldIndex(StructType.scala:233)
org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema.fieldIndex(rows.scala:213)
org.apache.spark.sql.Row$class.getAs(Row.scala:336)
org.apache.spark.sql.catalyst.expressions.GenericRow.getAs(rows.scala:192)
$line74.$read$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC.<init>(<console>:52)
$line74.$read$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC.<init>(<console>:57)
$line74.$read$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC.<init>(<console>:59)
$line74.$read$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC.<init>(<console>:61)
$line74.$read$$iwC$$iwC$

### Replace Now!

In [20]:
val trainWithoutNull = replacementFunction(train)
val testWithoutNull = replacementFunction(test)
val kaggleTestWithoutNull = replacementFunction(kaggleTest)

Replacing!
Replacing!
Replacing!


In [21]:
trainWithoutNull.rdd.getNumPartitions

4

In [22]:
trainWithoutNull.cache()
testWithoutNull.cache()
kaggleTestWithoutNull.cache()

[ID: int, v1: double, v2: double, v3: string, v4: double, v5: double, v6: double, v7: double, v8: double, v9: double, v10: double, v11: double, v12: double, v13: double, v14: double, v15: double, v16: double, v17: double, v18: double, v19: double, v20: double, v21: double, v22: string, v23: double, v24: string, v25: double, v26: double, v27: double, v28: double, v29: double, v30: string, v31: string, v32: double, v33: double, v34: double, v35: double, v36: double, v37: double, v38: int, v39: double, v40: double, v41: double, v42: double, v43: double, v44: double, v45: double, v46: double, v47: string, v48: double, v49: double, v50: double, v51: double, v52: string, v53: double, v54: double, v55: double, v56: string, v57: double, v58: double, v59: doub...

## Pipeline

In [23]:
// Create Label
val labelIndexer = new StringIndexer().setInputCol("target").setOutputCol("indexedLabel")

In [24]:
// Create Feature vector
val assembler = new VectorAssembler().setInputCols((columnsDict("IntegerType") ++ columnsDict("DoubleType")).toArray).setOutputCol("features")

In [25]:
val normalizer = new Normalizer().setInputCol("features").setOutputCol("normFeatures")

In [191]:
// Train a GBT model.
val gbt = new RandomForestClassifier().setFeaturesCol("normFeatures").setLabelCol("indexedLabel").setNumTrees(150).setMaxDepth(6)

In [192]:
// Chain indexer and GBT in a Pipeline
val pipeline = new Pipeline().setStages(Array(assembler, labelIndexer, normalizer, gbt))

In [193]:
// Train model.  This also runs the indexer.
val model = pipeline.fit(trainWithoutNull)

## Evaluation

In [194]:
println(model.stages(model.stages.length-1)) // summary only

RandomForestClassificationModel (uid=rfc_8e93302823a4) with 150 trees


In [195]:
val evaluator =  new MulticlassClassificationEvaluator().setLabelCol("indexedLabel").setPredictionCol("prediction").setMetricName("precision")
val evaluator =  new BinaryClassificationEvaluator().setLabelCol("indexedLabel").setRawPredictionCol("probability")

def evaluation(df:DataFrame) = {
    //df.stat.crosstab("indexedLabel", "prediction").show()
    
    //println(df.select("prediction", "indexedLabel", "probability").take(3)) // "rawPrediction")
    //println(rainPredictions.select("prediction", "indexedLabel", "normFeatures").take(3))
    //println("")
    
    val precision = evaluator.evaluate(df)
    println("Precision = " + precision)
}


### Training

In [196]:
// Make predictions.
val trainPredictions = model.transform(trainWithoutNull)
trainPredictions.cache()

[ID: int, target: int, v1: double, v2: double, v3: string, v4: double, v5: double, v6: double, v7: double, v8: double, v9: double, v10: double, v11: double, v12: double, v13: double, v14: double, v15: double, v16: double, v17: double, v18: double, v19: double, v20: double, v21: double, v22: string, v23: double, v24: string, v25: double, v26: double, v27: double, v28: double, v29: double, v30: string, v31: string, v32: double, v33: double, v34: double, v35: double, v36: double, v37: double, v38: int, v39: double, v40: double, v41: double, v42: double, v43: double, v44: double, v45: double, v46: double, v47: string, v48: double, v49: double, v50: double, v51: double, v52: string, v53: double, v54: double, v55: double, v56: string, v57: double, v58: double, ...

In [197]:
evaluation(trainPredictions)

Precision = 0.7159536152700261


In [213]:
val trainPredictionsShrink = shrinkDf(trainPredictions, None, 0.01)

In [214]:
println("Logloss on Training " + logloss(trainPredictionsShrink))

Logloss on Training 0.5045791727000289


In [200]:
trainPredictions.unpersist()

[ID: int, target: int, v1: double, v2: double, v3: string, v4: double, v5: double, v6: double, v7: double, v8: double, v9: double, v10: double, v11: double, v12: double, v13: double, v14: double, v15: double, v16: double, v17: double, v18: double, v19: double, v20: double, v21: double, v22: string, v23: double, v24: string, v25: double, v26: double, v27: double, v28: double, v29: double, v30: string, v31: string, v32: double, v33: double, v34: double, v35: double, v36: double, v37: double, v38: int, v39: double, v40: double, v41: double, v42: double, v43: double, v44: double, v45: double, v46: double, v47: string, v48: double, v49: double, v50: double, v51: double, v52: string, v53: double, v54: double, v55: double, v56: string, v57: double, v58: double, ...

### Testing

In [201]:
// Make predictions.
val testPredictions = model.transform(testWithoutNull)


In [202]:
evaluation(testPredictions)

Precision = 0.7037001165087784


In [211]:
val testPredictionsShrink = shrinkDf(testPredictions, None, 0.01)

In [212]:
println("Logloss on Testing " + logloss(testPredictionsShrink))

Logloss on Testing 0.5087857488568157


## Crossvalidation

In [40]:
// Grid search
// 5->30 * 2 with 5 folds, it takes 3 days -> precision 0.851998
//numTrees = 29
// maxDepth <= 10
val grid =  new ParamGridBuilder().addGrid(gbt.numTrees, 29 to 30).addGrid(gbt.maxDepth, 10 to 11).build()

In [41]:
// Cross validation
val cv = new CrossValidator().setEstimator(pipeline).setEstimatorParamMaps(grid).setEvaluator(evaluator).setNumFolds(5)

In [42]:
val modelCv = cv.fit(trainWithoutNull)
val predictionsCv = modelCv.transform(trainWithoutNull)

In [43]:
evaluation(predictionsCv)

+-----------------------+----+-----+
|indexedLabel_prediction| 1.0|  0.0|
+-----------------------+----+-----+
|                    1.0|1556|14820|
|                    0.0| 155|51948|
+-----------------------+----+-----+

[Lorg.apache.spark.sql.Row;@3eb4e2fb

Precision = 0.8307759015598617


### Testing best model

In [44]:
val testPredictionsCv = modelCv.transform(testWithoutNull)
evaluation(testPredictionsCv)

+-----------------------+---+-----+
|indexedLabel_prediction|1.0|  0.0|
+-----------------------+---+-----+
|                    1.0|243|10681|
|                    0.0|149|34769|
+-----------------------+---+-----+

[Lorg.apache.spark.sql.Row;@6bc01005

Precision = 0.7080827807615147


In [45]:
val testPredictionsShrinkCv = shrinkDf(testPredictionsCv, None, 0.01)

In [46]:
println("Logloss on Testing CV " + logloss(testPredictionsShrinkCv))

Logloss on Testing CV 0.5289056659818699


## Make prediction and save

In [215]:
val modelToSave =  model // or model modelCv

In [216]:
val predictions = model.transform(kaggleTestWithoutNull)

In [217]:
predictions.stat.crosstab("prediction", "prediction").show()

+---------------------+------+
|prediction_prediction|   0.0|
+---------------------+------+
|                  0.0|114393|
+---------------------+------+



In [218]:
val predictionsShrink = shrinkDf(predictions, None, 0.01)

In [219]:
val predictionsShrink.select("ID", "proba").take(3) // "probability", "rawPrediction"

Name: Compile Error
Message: <console>:1: error: '=' expected but '.' found.
       val predictionsShrink.select("ID", "proba").take(3) // "probability", "rawPrediction"
                                                  ^
StackTrace: 

In [220]:
val outputFile = "results/predictionScala.csv"
import sys.process._
"rm -rf " + outputFile !
predictionsShrink.select("ID", "proba").withColumnRenamed("proba", "PredictedProb").repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save(outputFile)

# Spark modifications
custom logloss metric

In [53]:
//package org.apache.spark.mllib.evaluation

import org.apache.spark.annotation.Since
import org.apache.spark.Logging
import org.apache.spark.mllib.evaluation.binary._
import org.apache.spark.rdd.{RDD, UnionRDD}
import org.apache.spark.sql.DataFrame

/**
 * Evaluator for binary classification.
 *
 * @param scoreAndLabels an RDD of (score, label) pairs.
 * @param numBins if greater than 0, then the curves (ROC curve, PR curve) computed internally
 *                will be down-sampled to this many "bins". If 0, no down-sampling will occur.
 *                This is useful because the curve contains a point for each distinct score
 *                in the input, and this could be as large as the input itself -- millions of
 *                points or more, when thousands may be entirely sufficient to summarize
 *                the curve. After down-sampling, the curves will instead be made of approximately
 *                `numBins` points instead. Points are made from bins of equal numbers of
 *                consecutive points. The size of each bin is
 *                `floor(scoreAndLabels.count() / numBins)`, which means the resulting number
 *                of bins may not exactly equal numBins. The last bin in each partition may
 *                be smaller as a result, meaning there may be an extra sample at
 *                partition boundaries.
 */
@Since("1.0.0")
class BinaryClassificationMetrics @Since("1.3.0") (
    @Since("1.3.0") val scoreAndLabels: RDD[(Double, Double)],
    @Since("1.3.0") val numBins: Int) extends Logging {

  require(numBins >= 0, "numBins must be nonnegative")

  /**
   * Defaults `numBins` to 0.
   */
  @Since("1.0.0")
  def this(scoreAndLabels: RDD[(Double, Double)]) = this(scoreAndLabels, 0)

  /**
   * An auxiliary constructor taking a DataFrame.
   * @param scoreAndLabels a DataFrame with two double columns: score and label
   */
  private[mllib] def this(scoreAndLabels: DataFrame) =
    this(scoreAndLabels.rdd.map(r => (r.getDouble(0), r.getDouble(1))))

  /**
   * Unpersist intermediate RDDs used in the computation.
   */
  @Since("1.0.0")
  def unpersist() {
    cumulativeCounts.unpersist()
  }

  /**
   * Returns thresholds in descending order.
   */
  @Since("1.0.0")
  def thresholds(): RDD[Double] = cumulativeCounts.map(_._1)

  /**
   * Returns the receiver operating characteristic (ROC) curve,
   * which is an RDD of (false positive rate, true positive rate)
   * with (0.0, 0.0) prepended and (1.0, 1.0) appended to it.
   * @see http://en.wikipedia.org/wiki/Receiver_operating_characteristic
   */
  @Since("1.0.0")
  def roc(): RDD[(Double, Double)] = {
    val rocCurve = createCurve(FalsePositiveRate, Recall)
    val sc = confusions.context
    val first = sc.makeRDD(Seq((0.0, 0.0)), 1)
    val last = sc.makeRDD(Seq((1.0, 1.0)), 1)
    new UnionRDD[(Double, Double)](sc, Seq(first, rocCurve, last))
  }

  /**
   * Computes the area under the receiver operating characteristic (ROC) curve.
   */
  @Since("1.0.0")
  def areaUnderROC(): Double = AreaUnderCurve.of(roc())
    
  /**
   * Computes the area under the receiver operating characteristic (ROC) curve.
   */
  @Since("1.0.0")
  def logloss(): Double = {
    var loglossRed1 = scoreAndLabels //df.map(r => (r.getAs[Double]("indexedLabel"), r.getAs[Double]("proba"))) //probability[1]
    
    // Check if(some proba are < 0
    val neg = loglossRed1.filter{ case (p,y) => (p <= 0.0 || p >= 1.0)}
    val negCount = neg.count()
    if(negCount != 0) {
        println("!!! There so non-valid probability !!! " + negCount)
        loglossRed1 = loglossRed1.filter{ case (p,y) => (p > 0.0 && p < 1.0)}
    }
    
    val loglossRed2 =  loglossRed1.map{case (p,y) => y*log(p) + (1.0-y)*log(1.0-p)}
    val loglossRed  =  loglossRed2.reduce((a, b) => a+b)
    
    return -1.0 * loglossRed / df.count()
  }

  /**
   * Returns the precision-recall curve, which is an RDD of (recall, precision),
   * NOT (precision, recall), with (0.0, 1.0) prepended to it.
   * @see http://en.wikipedia.org/wiki/Precision_and_recall
   */
  @Since("1.0.0")
  def pr(): RDD[(Double, Double)] = {
    val prCurve = createCurve(Recall, Precision)
    val sc = confusions.context
    val first = sc.makeRDD(Seq((0.0, 1.0)), 1)
    first.union(prCurve)
  }

  /**
   * Computes the area under the precision-recall curve.
   */
  @Since("1.0.0")
  def areaUnderPR(): Double = AreaUnderCurve.of(pr())

  /**
   * Returns the (threshold, F-Measure) curve.
   * @param beta the beta factor in F-Measure computation.
   * @return an RDD of (threshold, F-Measure) pairs.
   * @see http://en.wikipedia.org/wiki/F1_score
   */
  @Since("1.0.0")
  def fMeasureByThreshold(beta: Double): RDD[(Double, Double)] = createCurve(FMeasure(beta))

  /**
   * Returns the (threshold, F-Measure) curve with beta = 1.0.
   */
  @Since("1.0.0")
  def fMeasureByThreshold(): RDD[(Double, Double)] = fMeasureByThreshold(1.0)

  /**
   * Returns the (threshold, precision) curve.
   */
  @Since("1.0.0")
  def precisionByThreshold(): RDD[(Double, Double)] = createCurve(Precision)

  /**
   * Returns the (threshold, recall) curve.
   */
  @Since("1.0.0")
  def recallByThreshold(): RDD[(Double, Double)] = createCurve(Recall)

  private lazy val (
    cumulativeCounts: RDD[(Double, BinaryLabelCounter)],
    confusions: RDD[(Double, BinaryConfusionMatrix)]) = {
    // Create a bin for each distinct score value, count positives and negatives within each bin,
    // and then sort by score values in descending order.
    val counts = scoreAndLabels.combineByKey(
      createCombiner = (label: Double) => new BinaryLabelCounter(0L, 0L) += label,
      mergeValue = (c: BinaryLabelCounter, label: Double) => c += label,
      mergeCombiners = (c1: BinaryLabelCounter, c2: BinaryLabelCounter) => c1 += c2
    ).sortByKey(ascending = false)

    val binnedCounts =
      // Only down-sample if bins is > 0
      if (numBins == 0) {
        // Use original directly
        counts
      } else {
        val countsSize = counts.count()
        // Group the iterator into chunks of about countsSize / numBins points,
        // so that the resulting number of bins is about numBins
        var grouping = countsSize / numBins
        if (grouping < 2) {
          // numBins was more than half of the size; no real point in down-sampling to bins
          logInfo(s"Curve is too small ($countsSize) for $numBins bins to be useful")
          counts
        } else {
          if (grouping >= Int.MaxValue) {
            logWarning(
              s"Curve too large ($countsSize) for $numBins bins; capping at ${Int.MaxValue}")
            grouping = Int.MaxValue
          }
          counts.mapPartitions(_.grouped(grouping.toInt).map { pairs =>
            // The score of the combined point will be just the first one's score
            val firstScore = pairs.head._1
            // The point will contain all counts in this chunk
            val agg = new BinaryLabelCounter()
            pairs.foreach(pair => agg += pair._2)
            (firstScore, agg)
          })
        }
      }

    val agg = binnedCounts.values.mapPartitions { iter =>
      val agg = new BinaryLabelCounter()
      iter.foreach(agg += _)
      Iterator(agg)
    }.collect()
    val partitionwiseCumulativeCounts =
      agg.scanLeft(new BinaryLabelCounter())(
        (agg: BinaryLabelCounter, c: BinaryLabelCounter) => agg.clone() += c)
    val totalCount = partitionwiseCumulativeCounts.last
    logInfo(s"Total counts: $totalCount")
    val cumulativeCounts = binnedCounts.mapPartitionsWithIndex(
      (index: Int, iter: Iterator[(Double, BinaryLabelCounter)]) => {
        val cumCount = partitionwiseCumulativeCounts(index)
        iter.map { case (score, c) =>
          cumCount += c
          (score, cumCount.clone())
        }
      }, preservesPartitioning = true)
    cumulativeCounts.persist()
    val confusions = cumulativeCounts.map { case (score, cumCount) =>
      (score, BinaryConfusionMatrixImpl(cumCount, totalCount).asInstanceOf[BinaryConfusionMatrix])
    }
    (cumulativeCounts, confusions)
  }

  /** Creates a curve of (threshold, metric). */
  private def createCurve(y: BinaryClassificationMetricComputer): RDD[(Double, Double)] = {
    confusions.map { case (s, c) =>
      (s, y(c))
    }
  }

  /** Creates a curve of (metricX, metricY). */
  private def createCurve(
      x: BinaryClassificationMetricComputer,
      y: BinaryClassificationMetricComputer): RDD[(Double, Double)] = {
    confusions.map { case (_, c) =>
      (x(c), y(c))
    }
  }
}

Name: Compile Error
Message: <console>:1: error: illegal start of definition
       package org.apache.spark.mllib.evaluation
       ^
StackTrace: 

In [54]:
//package org.apache.spark.ml.evaluation

import org.apache.spark.annotation.{Experimental, Since}
import org.apache.spark.ml.param._
import org.apache.spark.ml.param.shared._
import org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable, Identifiable, SchemaUtils}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.linalg.{Vector, VectorUDT}
import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.types.DoubleType

/**
 * :: Experimental ::
 * Evaluator for binary classification, which expects two input columns: rawPrediction and label.
 * The rawPrediction column can be of type double (binary 0/1 prediction, or probability of label 1)
 * or of type vector (length-2 vector of raw predictions, scores, or label probabilities).
 */
@Since("1.2.0")
@Experimental
class BinaryClassificationEvaluator @Since("1.4.0") (@Since("1.4.0") override val uid: String)
  extends Evaluator with HasRawPredictionCol with HasLabelCol with DefaultParamsWritable {

  @Since("1.2.0")
  def this() = this(Identifiable.randomUID("binEval"))

  /**
   * param for metric name in evaluation (supports `"areaUnderROC"` (default), `"areaUnderPR"`)
   * @group param
   */
  @Since("1.2.0")
  val metricName: Param[String] = {
    val allowedParams = ParamValidators.inArray(Array("areaUnderROC", "areaUnderPR", "logloss"))
    new Param(
      this, "metricName", "metric name in evaluation (areaUnderROC|areaUnderPR|logloss)", allowedParams)
  }

  /** @group getParam */
  @Since("1.2.0")
  def getMetricName: String = $(metricName)

  /** @group setParam */
  @Since("1.2.0")
  def setMetricName(value: String): this.type = set(metricName, value)

  /** @group setParam */
  @Since("1.5.0")
  def setRawPredictionCol(value: String): this.type = set(rawPredictionCol, value)

  /**
   * @group setParam
   * @deprecated use [[setRawPredictionCol()]] instead
   */
  @deprecated("use setRawPredictionCol instead", "1.5.0")
  @Since("1.2.0")
  def setScoreCol(value: String): this.type = set(rawPredictionCol, value)

  /** @group setParam */
  @Since("1.2.0")
  def setLabelCol(value: String): this.type = set(labelCol, value)

  setDefault(metricName -> "areaUnderROC")

  @Since("1.2.0")
  override def evaluate(dataset: DataFrame): Double = {
    val schema = dataset.schema
    SchemaUtils.checkColumnTypes(schema, $(rawPredictionCol), Seq(DoubleType, new VectorUDT))
    SchemaUtils.checkColumnType(schema, $(labelCol), DoubleType)

    // TODO: When dataset metadata has been implemented, check rawPredictionCol vector length = 2.
    val scoreAndLabels = dataset.select($(rawPredictionCol), $(labelCol)).rdd.map {
      case Row(rawPrediction: Vector, label: Double) => (rawPrediction(1), label)
      case Row(rawPrediction: Double, label: Double) => (rawPrediction, label)
    }
    val metrics = new BinaryClassificationMetrics(scoreAndLabels)
    val metric = $(metricName) match {
      case "areaUnderROC" => metrics.areaUnderROC()
      case "areaUnderPR" => metrics.areaUnderPR()
      case "logloss"  => metrics.logloss()
    }
    metrics.unpersist()
    metric
  }

  @Since("1.5.0")
  override def isLargerBetter: Boolean = $(metricName) match {
    case "areaUnderROC" => true
    case "areaUnderPR" => true
    case "logloss" => true
  }

  @Since("1.4.1")
  override def copy(extra: ParamMap): BinaryClassificationEvaluator = defaultCopy(extra)
}

@Since("1.6.0")
object BinaryClassificationEvaluator extends DefaultParamsReadable[BinaryClassificationEvaluator] {

  @Since("1.6.0")
  override def load(path: String): BinaryClassificationEvaluator = super.load(path)
}

Name: Compile Error
Message: <console>:1: error: illegal start of definition
       package org.apache.spark.ml.evaluation
       ^
StackTrace: 